In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from pykrx import stock

import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)

In [ ]:
# KOSPI200 지수 데이터 불러오기

In [ ]:
kospi200 = yf.download("^KS200", start="2020-01-01", end="2025-01-01")
kospi200.columns=kospi200.columns.droplevel(level=1)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [30]:
# 분석할 기간 설정
start = kospi200.index[0]
end = kospi200.index[-1]

# 매달 둘째 주 목요일 추출
dates = pd.date_range(start=start, end=end, freq="W-THU")  # 모든 목요일
second_thu = dates.to_series().groupby([dates.year, dates.month]).nth(1)  # 각 월별 2번째 목요일
thu_list = pd.DatetimeIndex(second_thu.to_list())

In [ ]:
kospi200_selected = kospi200.loc[kospi200.index.intersection(thu_list)] # 실제 인덱스 교집합만 사용

In [32]:
kospi200_selected

Price,Close,High,Low,Open,Volume
2020-01-09,294.410004,294.410004,292.100006,293.829987,111800000
2020-02-13,300.929993,304.429993,300.480011,302.790009,115100000
2020-03-12,247.619995,255.490005,243.759995,253.929993,242500000
2020-04-09,245.610001,246.460007,243.110001,246.130005,175900000
2020-05-14,253.649994,254.389999,252.089996,253.259995,167600000
2020-06-11,288.619995,292.079987,284.290009,289.980011,275900000
2020-07-09,287.250000,289.410004,287.079987,287.799988,140600000
2020-08-13,323.329987,326.220001,319.660004,325.839996,285700000
2020-09-10,316.529999,318.380005,315.459991,317.920013,234300000
2020-10-08,317.470001,320.380005,316.600006,320.309998,168700000
